In [12]:
from elasticsearch import Elasticsearch
import json
from bson import ObjectId
from elasticsearch import helpers
import numpy as np

In [3]:
es_client = Elasticsearch('https://localhost:9200', basic_auth=("elastic", "6E0GWL_MEddnKJWCnk*M"),
                    ca_certs="./http_ca.crt")

In [ ]:
# start = time.time()
# text = request.args.get('text')

# suggest_dictionary = {
#     "text": text,
#     "autocomplete": {"term": {"field": "Description"}},
# }
# query_dictionary = {"suggest": suggest_dictionary}
# res = app.es_client.search(index="recipe", body=query_dictionary)
# resp = list(res["suggest"]["autocomplete"])
# result = []

# for word in resp:
#     text = {'text': word['text'], 'candidates': word['options']}
#     result.append(text)

# end = time.time()
# response = {'elapse': end - start, 'result': result, 'status': 'success'}

# return response

In [38]:
suggest_dictionary = {
    "text": "pancoke blue berrt",
    "autocomplete-1": {"term": {"field": "Description"}},
    "autocomplete-2": {"term": {"field": "Name"}},
    "autocomplete-3": {"term": {"field": "RecipeInstructions"}},
}

query_dictionary = {"suggest": suggest_dictionary}
res = es_client.search(index="recipe", body=query_dictionary)

In [46]:
from collections import defaultdict

def combine_autocomplete_terms(autocomplete_terms):
    combined_results = []

    for term in autocomplete_terms:
        result = {"text": term[0]["text"], "candidates": defaultdict(lambda: {"score": 0, "freq": 0})}

        for option in term:
            if option["options"]:
                for candidate in option["options"]:
                    candidate_text = candidate["text"]
                    candidate_score = candidate["score"]
                    candidate_freq = candidate["freq"]
                    existing_candidate = result["candidates"][candidate_text]
                    
                    # Update weighted average score and frequency
                    existing_candidate["score"] = (
                        (existing_candidate["score"] * existing_candidate["freq"]) + 
                        (candidate_score * candidate_freq)
                    ) / (existing_candidate["freq"] + candidate_freq)
                    existing_candidate["freq"] += candidate_freq

        # Convert defaultdict to regular dictionary
        result["candidates"] = dict(result["candidates"])
        combined_results.append(result)

    return combined_results



In [48]:
p = []

for term in np.array(list(res["suggest"].values())).T:
    result = {}
    result["text"] = term[0]["text"]
    options = [v["options"] for v in term]
    result["candidates"] = {}
    for option in options:
        candidates = {}
        if len(option) > 0:
            candidates["text"] = option[0]["text"]
            for candidate in option:
                if candidate["text"] not in result["candidates"]:
                    result["candidates"][candidate["text"]] = {
                        "score": candidate["score"],
                        "freq": candidate["freq"],
                    }
                else:
                    result["candidates"][candidate["text"]]["score"] = (
                        result["candidates"][candidate["text"]]["score"]
                        * result["candidates"][candidate["text"]]["freq"]
                        + candidate["score"] * candidate["freq"]
                    ) / (
                        result["candidates"][candidate["text"]]["freq"]
                        + candidate["freq"]
                    )
                    result["candidates"][candidate["text"]]["freq"] = (
                        result["candidates"][candidate["text"]]["freq"]
                        + candidate["freq"]
                    )
    p += [result["candidates"]] 


In [49]:
p

[{'pancake': {'score': 0.85714287, 'freq': 298},
  'pancakes': {'score': 0.71428573, 'freq': 628},
  'panckes': {'score': 0.71428573, 'freq': 1},
  'panco': {'score': 0.6, 'freq': 1}},
 {},
 {'berry': {'score': 0.8000000000000002, 'freq': 119},
  'burst': {'score': 0.6, 'freq': 34},
  'burnt': {'score': 0.5999999999999999, 'freq': 33},
  'bears': {'score': 0.6, 'freq': 4},
  'beast': {'score': 0.6, 'freq': 3},
  'beard': {'score': 0.6, 'freq': 2},
  'beery': {'score': 0.6, 'freq': 1},
  'beers': {'score': 0.6, 'freq': 2},
  'barry': {'score': 0.6, 'freq': 1}}]

In [52]:
resp = []

for suggestion in p:
    new_data = [{'text': key, 'score': value['score'], 'freq': value['freq']} for key, value in suggestion.items()]
    resp.append(new_data)




In [53]:
resp

[[{'text': 'pancake', 'score': 0.85714287, 'freq': 298},
  {'text': 'pancakes', 'score': 0.71428573, 'freq': 628},
  {'text': 'panckes', 'score': 0.71428573, 'freq': 1},
  {'text': 'panco', 'score': 0.6, 'freq': 1}],
 [],
 [{'text': 'berry', 'score': 0.8000000000000002, 'freq': 119},
  {'text': 'burst', 'score': 0.6, 'freq': 34},
  {'text': 'burnt', 'score': 0.5999999999999999, 'freq': 33},
  {'text': 'bears', 'score': 0.6, 'freq': 4},
  {'text': 'beast', 'score': 0.6, 'freq': 3},
  {'text': 'beard', 'score': 0.6, 'freq': 2},
  {'text': 'beery', 'score': 0.6, 'freq': 1},
  {'text': 'beers', 'score': 0.6, 'freq': 2},
  {'text': 'barry', 'score': 0.6, 'freq': 1}]]